In [1]:
import vaex
import geopandas as gpd
import pandas as pd
from difflib import SequenceMatcher

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
dfs = []

for i in range(1995,2023):
    dfs.append(vaex.open(f'data/IPTU_{i}/IPTU_{i}.hdf5'))

In [3]:
df_iptu = vaex.concat(dfs)

In [4]:
len(df_iptu)

82322059

In [5]:
df_iptu.groupby('ANO DO EXERCICIO').agg('count')

#,ANO DO EXERCICIO,count
0,1995,2356613
1,1996,2389535
2,1997,2422173
3,1998,2467107
4,1999,2497796
...,...,...
23,2018,3414208
24,2019,3447112
25,2020,3498644
26,2021,3489273


In [6]:
df_usos_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()
df_usos_2022

,TIPO DE USO DO IMOVEL,count
0,Residência e outro uso (predominância residenc...,85643
1,"Outras edificações de uso comercial, com utili...",22163
2,Posto de serviço,2279
3,"Prédio de apartamento, não em condomínio, de u...",2209
4,"Estação radioemissora, de televisão ou empresa...",495
5,Flat de uso comercial (semelhante a hotel),24619
6,Apartamento em condomínio,1439450
7,Garagem (unidade autônoma) em edifício em cond...,200254
8,Clube esportivo,573
9,"Outras edificações de uso especial, com utiliz...",1263


In [7]:
df_padroes_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE PADRAO DA CONSTRUCAO').agg('count').to_pandas_df()
df_padroes_2022

,TIPO DE PADRAO DA CONSTRUCAO,count
0,Residencial horizontal - padrão A,285433
1,Oficina/Posto de serviço/Armazém/D,21814
2,Residencial vertical - padrão E,56532
3,Residencial horizontal - padrão F,2022
4,Barracão/Telheiro/Oficina - padrão,4067
5,Residencial horizontal - padrão B,395050
6,Residencial horizontal - padrão E,8779
7,Comercial vertical - padrão B,54602
8,Residencial horizontal - padrão D,47294
9,TERRENO,103197


In [8]:
df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE USO DO IMOVEL').agg('count')

#,TIPO DE USO DO IMOVEL,count
0,Residência e outro uso (predominância residencial),85643
1,"'Outras edificações de uso comercial, com utiliz...",22163
2,Posto de serviço,2279
3,"'Prédio de apartamento, não em condomínio, de us...",2209
4,"'Estação radioemissora, de televisão ou empresa ...",495
...,...,...
31,Terreno,103197
32,"Asilo, orfanato, creche, seminário ou convento",989
33,Garagem (unidade autônoma) de prédio de garagens,9568
34,"'Prédio de escritório ou consultório, não em con...",24185


In [9]:
df_usos = df_iptu.groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()
# df_usos = pd.read_csv('criterios/tipos-de-uso-do-imovel.csv')

In [10]:
df_usos.fillna('', inplace=True)

In [11]:
results, ratios = [], []

for i, row in df_usos.iterrows():

    similaridade = df_usos_2022['TIPO DE USO DO IMOVEL'].apply(lambda x: SequenceMatcher(None, row['TIPO DE USO DO IMOVEL'], x).ratio())
    
    results.append(df_usos_2022.iloc[similaridade.idxmax()]['TIPO DE USO DO IMOVEL'])

    ratios.append(similaridade.max())


In [12]:
df_usos['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_usos['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [13]:
residencial = ['residencial', 'residência', 'apartamento', 'cortiço']
comercial = ['comércio', 'comercial', 'posto', 'loja']
servicos =  ['hotel', 'radioemissora', 'flat', 'serviço', 'clube', 'oficina', 'hospital', 'templo', 'escritórios', 'consultórios', 'escritório', 'consultório', 'asilo', 'creche', 'prédio de garagens', 'escola']
industrial = ['Indústria', 'armazéns']

In [14]:
# df_usos['classificacao_abrangente'] = '' # Para definir entre: residencial, comercial/servicos, industrial, outros
class_residencial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(residencial)), case=False)
class_comercial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(comercial)), case=False)
class_servicos = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(servicos)), case=False)
class_industrial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(industrial)), case=False)

df_usos.loc[class_residencial, ['classificacao_abrangente']] = 'Residencial'
df_usos.loc[class_comercial, ['classificacao_abrangente']] = 'Comercial'
df_usos.loc[class_servicos, ['classificacao_abrangente']] = 'Serviços'
df_usos.loc[class_industrial, ['classificacao_abrangente']] = 'Industrial'

df_usos.loc[df_usos.classificacao_abrangente.isna(), ['classificacao_abrangente']] = 'Outros'


In [15]:
df_usos.to_csv('criterios/tipos-de-uso-do-imovel-padronizados.csv')

In [16]:
df_usos

,TIPO DE USO DO IMOVEL,count,titulo_padronizado,relacao_de_assertividade_com_titulo_padronizado,classificacao_abrangente
0,,26,Residência e outro uso (predominância residenc...,0.000000,Residencial
1,"Outras edificações de uso comercial, com utili...",147516,"Outras edificações de uso comercial, com utili...",1.000000,Comercial
2,Residência e outro uso (predominância residenc...,595522,Residência e outro uso (predominância residenc...,1.000000,Residencial
3,"outras edificações do tipo (uso especial), com...",24600,"Outras edificações de uso especial, com utiliz...",0.912000,Outros
4,Garagem (unidade autônoma) em edifício em cond...,178032,Garagem (unidade autônoma) em edifício em cond...,0.895706,Serviços
...,...,...,...,...,...
73,"asilo, orfanato, creche, seminário ou convento",11795,"Asilo, orfanato, creche, seminário ou convento",0.978261,Serviços
74,cortiço (habitação coletiva subnormal),41620,Residência e outro uso (predominância residenc...,0.363636,Residencial
75,escola,135829,Escola,0.833333,Serviços
76,"Prédio de escritório ou consultório, não em co...",173977,"Prédio de escritório ou consultório, não em co...",1.000000,Serviços


In [17]:
df_padroes = df_iptu.groupby('TIPO DE PADRAO DA CONSTRUCAO').agg('count').to_pandas_df()

In [18]:
df_padroes.fillna('', inplace=True)

In [19]:
horizontal = df_padroes['TIPO DE PADRAO DA CONSTRUCAO'].str.contains('horizontal', case=False)
vertical = df_padroes['TIPO DE PADRAO DA CONSTRUCAO'].str.contains('vertical', case=False)
terreno = df_padroes['TIPO DE PADRAO DA CONSTRUCAO'].str.contains('terreno', case=False)

In [20]:
padroes = df_padroes['TIPO DE PADRAO DA CONSTRUCAO'].str[-1].isin([l for l in 'ABCDEF'])
df_padroes.loc[padroes, 'padrao'] = df_padroes.loc[padroes, 'TIPO DE PADRAO DA CONSTRUCAO'].str[-1]

In [21]:
df_padroes.loc[vertical, 'contrucao'] = 'vertical'
df_padroes.loc[horizontal, 'contrucao'] = 'horizontal'
df_padroes.loc[terreno, 'contrucao'] = 'terreno'

In [22]:
df_padroes.fillna('', inplace=True)

In [23]:
results, ratios = [], []

for i, row in df_padroes.iterrows():

    similaridade = df_padroes_2022['TIPO DE PADRAO DA CONSTRUCAO'].apply(lambda x: SequenceMatcher(None, row['TIPO DE PADRAO DA CONSTRUCAO'], x).ratio())
    
    results.append(df_padroes_2022.iloc[similaridade.idxmax()]['TIPO DE PADRAO DA CONSTRUCAO'])

    ratios.append(similaridade.max())


In [24]:
df_padroes['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_padroes['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [25]:
df_padroes.to_csv('criterios/tipos-de-padrao-construcao-padronizados.csv')